## Import dependencies

Make sure you are in the wind_gust conda environment

Basemap is deprecated. Works in this environment because python was downgraded to 3.8.10. Cartopy provides similar features may be used in the future. 

In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.inspection import plot_partial_dependence
import pickle

## Sample data

More data will be added here in the future, spanning different seasons. 

In [ ]:
Met_20170701 = Dataset(  # Metar Training Data
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/20170701_0000')
Met_20180613 = Dataset(  # Metar Training Data
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/20180613_0000')
Met_20190801 = Dataset( # Metar Testing Data
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/2019_08_01/20190801_0300')
Hrrr = Dataset(  # HRRR
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/extract2Da.esrl_hrrr.201718201001300.nc')

#Met_20170701.variables
#Met_20170701.variables['windGust']
#Hrrr.variables['GUST_P0_L1_GLC0']

# Format metar data

### Compile and crop metar data to the continental US

There is likely a more efficient way to do this. The warning of "converting a masked element to nan" is currently expected, but may be changed to do manually in the future. 

In [ ]:
met_stationName_1 = Met_20170701.variables['stationName']
met_locationName_1 = Met_20170701.variables['locationName']
met_qct_1 = Met_20170701.variables['QCT']
met_lon_1 = Met_20170701.variables['longitude']
met_lat_1 = Met_20170701.variables['latitude']
met_elevation_1 = Met_20170701.variables['elevation']
met_timeObs_1 = Met_20170701.variables['timeObs']
met_skyCover_1 = Met_20170701.variables['skyCover']
met_seaLevelPress_1 = Met_20170701.variables['seaLevelPress']
met_altimeter_1 = Met_20170701.variables['altimeter']
met_pressChange3Hour_1 = Met_20170701.variables['pressChange3Hour']

met_gust_1 = Met_20170701.variables['windGust']
met_temp_1 = Met_20170701.variables['temperature']
met_windspeed_1 = Met_20170701.variables['windSpeed']
met_winddir_1 = Met_20170701.variables['windDir']
met_press_1 = Met_20170701.variables['seaLevelPress']

met_gust_2 = Met_20180613.variables['windGust']
met_lon_2 = Met_20180613.variables['longitude']
met_lat_2 = Met_20180613.variables['latitude']
met_temp_2 = Met_20180613.variables['temperature']
met_windspeed_2 = Met_20180613.variables['windSpeed']
met_winddir_2 = Met_20180613.variables['windDir']
met_press_2 = Met_20180613.variables['seaLevelPress']

#--------Test Data--------
met_gust_test = Met_20190801.variables['windGust']
met_lon_test = Met_20190801.variables['longitude']
met_lat_test = Met_20190801.variables['latitude']
met_temp_test = Met_20190801.variables['temperature']
met_windspeed_test = Met_20190801.variables['windSpeed']
met_winddir_test = Met_20190801.variables['windDir']
met_press_test = Met_20190801.variables['seaLevelPress']

top = 49.3457868  # north lat
left = -124.7844079  # west lon
right = -66.9513812  # east lon
bottom = 24.7433195  # south lat

cell = []
for g, lon, lat, t, ws, wd, p in zip(met_gust_1, met_lon_1, met_lat_1, 
                                     met_temp_1, met_windspeed_1, met_winddir_1, met_press_1):
    if left <= lon <= right and bottom <= lat <= top:
        cell.append((g, lon, lat, t, ws, wd, p))

for g, lon, lat, t, ws, wd, p in zip(met_gust_2, met_lon_2, met_lat_2, 
                                     met_temp_2, met_windspeed_2, met_winddir_2, met_press_2):
    if left <= lon <= right and bottom <= lat <= top:
        cell.append((g, lon, lat, t, ws, wd, p))

met_pos = np.asarray(cell)

#--------Test Data--------
cell_test = []
for g, lon, lat, t, ws, wd, p in zip(met_gust_test, met_lon_test, met_lat_test, 
                                     met_temp_test, met_windspeed_test, met_winddir_test, met_press_test):
    if left <= lon <= right and bottom <= lat <= top:
        cell_test.append((g, lon, lat, t, ws, wd, p))

met_pos_test = np.asarray(cell_test)

### Convert metar data to dataframe

In [ ]:
met_df = pd.DataFrame(
    {'MetWindGust': met_pos[:, 0],
     'Longitude': met_pos[:, 1],
     'Latitude': met_pos[:, 2],
     'Temperature': met_pos[:, 3],
     'WindSpeed': met_pos[:, 4],
     'WindDirection': met_pos[:, 5],
     'Pressure': met_pos[:, 6]})

met_df = met_df.round(4)
met_df = met_df.replace(np.NaN, 0)
pd.set_option('display.min_rows', 100)
met_df

#--------Test Data--------
met_df_test = pd.DataFrame(
    {'MetWindGust': met_pos_test[:, 0],
     'Longitude': met_pos_test[:, 1],
     'Latitude': met_pos_test[:, 2],
     'Temperature': met_pos_test[:, 3],
     'WindSpeed': met_pos_test[:, 4],
     'WindDirection': met_pos_test[:, 5],
     'Pressure': met_pos_test[:, 6]})

met_df_test = met_df_test.round(4)
met_df_test = met_df_test.replace(np.NaN, 0)

### Plot metar locations

Uses the converted zeroes from the dataframe. 

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(met_df.Longitude, met_df.Latitude, latlon=True, c=met_df.MetWindGust,
          cmap='Purples', vmin=0, vmax=10, s=5)

plt.colorbar(label='Measured gusts (meter/sec)', extend='max')

plt.show()

### Rescaling inputs, splitting train and test

Want to still be able to plot the latitude and longitude. 

In [ ]:
array = met_df.values
X = array[:, 1:8]
y = array[:, 0]

#--------Test Data--------
array_test = met_df_test.values
X_test = array_test[:, 1:8]
y_test = array_test[:, 0]

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = X
y_train = y


scaler = StandardScaler()
rescaled_X_train = (scaler.fit_transform(X_train[:, 3:8])) # doesn't include lon, lat
rescaled_X_test = (scaler.fit_transform(X_test[:, 3:8])) # doesn't include lon, lat

### Training and running neural net

Currently, predictions are made with the data of the same time. Inputs are scaled so they have comparable impact. Standard Scaler: z = (x - u)/s. 

u = mean, s = standard deviation.

Two hidden layers: 100, 50. Used some rudimentary judging to come to this, testing and evaluating with RMSE. Read this for more info on the optimal amount: https://www.yourdatateacher.com/2021/05/10/how-many-neurons-for-a-neural-network/

Activation function: Currently using rectified linear activation function (relu), has a flat-then linear shape, will allow for more dynamic range. Will output the input directly if it is positive, otherwise, it will output zero.

Solver: adam. Uses back-propagration. 

Max Iterations: 1500. May be a little much. 

The model is saved as mlp_wind_gust.

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001,
                   power_t=0.5, max_iter=1500, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp = mlp.fit(rescaled_X_train, y_train)

pickle.dump(mlp, open('mlp_wind_gust.pkl','wb'))

y_predict_mlp = mlp.predict(rescaled_X_test)

Rounding any predictions < 5 to zero, to match the metar process. Metar does not measure gusts less than 5 m/s.

In [ ]:
for i in range(len(y_predict_mlp)):
    if y_predict_mlp[i] < 5:
        y_predict_mlp[i] = 0

### Plot neural net predictions

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(X_test[:, 0], X_test[:, 1], latlon=True, c=y_predict_mlp, cmap='Purples',
          vmin=0, vmax=10, s=5)

plt.colorbar(label='predicted gusts (meter/sec)', extend='max')

plt.show()

### Evaluating neural net predictions

Root Mean Square Error, along with other interesting features of the predictions. 

Future: Add AUROC (Area Under the Curve of the Receiver Operating Characteristic)

In [ ]:
print('RMSE:', np.sqrt(
    metrics.mean_squared_error(y_test, y_predict_mlp)).round(4))
print('Mean Absolute Error:', 
    metrics.mean_absolute_error(y_test, y_predict_mlp).round(4))
print('R-Squared Score:', 
    metrics.r2_score(y_test, y_predict_mlp).round(4))

print('Range of Training: ', np.min(y_train), ' to ', np.max(y_train))
print('Range of Test: ', np.min(y_test), ' to ', np.max(y_test))
print('Range of Predictions: ', np.min(y_predict_mlp).round(4), ' to ', np.max(y_predict_mlp).round(4))

In [ ]:
plt.scatter(y_predict_mlp, y_test, c='Navy', s=20)
plt.ylabel("Metar Gust (m/s)", size = 10)
plt.xlabel("Neural Network Gust (m/s)", size = 10)

### Loss Curve

Loss value evaluated at the end of each training step.

In [ ]:
plt.plot(mlp.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()
print('Loss:', mlp.loss_.round(4))
print('Best Loss:', mlp.best_loss_.round(4))
print('# of Training Samples:', mlp.t_)
print('# of Iterations:', mlp.n_iter_)
print('# of Layers:', mlp.n_layers_)
print('# of Outputs:', mlp.n_outputs_)
print('Output Activation Function:', mlp.out_activation_)

### Coefficients
The ith element in the list represents the weight matrix corresponding to layer i (scikit-learn docs).
Plots the weights for each neuron of each layer.

In [ ]:
plt.plot(mlp.coefs_[0]) #Also 1, 2
plt.title("Coefficients", fontsize=14)
plt.xlabel('Neuron')
plt.ylabel('Weights')
plt.show()

### Intercepts
The ith element in the list represents the bias vector corresponding to layer i + 1 (scikit-learn docs).  

In [ ]:
plt.plot(mlp.intercepts_[0]) #Also 1, 2
plt.title("Intercepts", fontsize=14)
plt.xlabel('Neuron')
plt.ylabel('Bias')
plt.show()

### Histograms

Comparing the frequencies of wind-gust speeds between the METAR and the neural net.

In [ ]:
hist_metar = plt.hist(y_test, bins=30, color='navy', range=[0,30], )
plt.title("METAR Readings")
plt.ylim(0,300)

In [ ]:
hist_nn = plt.hist(y_predict_mlp, bins=30, color='orange', range=[0,30])
plt.title("Neural Net Predictions")
plt.ylim(0,300)

In [ ]:
plt.scatter(X_test[:, 0], y_test,  color='navy', s=5, alpha=1)
plt.scatter(X_test[:, 0], y_predict_mlp, color='orange', s=5, alpha=0.3)
plt.title("Scatter Comparison")
plt.xlabel("Longitude", size = 10)
plt.ylabel("Wind Gust (meter/sec)", size = 10)

### Partial dependence plots
I don't understand these. Possibly need different features. 

In [ ]:
# features = [0, 1, 2, 3]

# display = plot_partial_dependence(
#      mlp, rescaled_X_train, features, kind="both", subsample=50,
#      n_jobs=3, grid_resolution=20, random_state=0
# )

## Training neural net with only measured wind gusts
Wind gusts of zero are not included in training. 

Uses the same variables as the above, so make sure to clear before re-running any lines outside of this section.

In [ ]:
subset_met_df = met_df[met_df["MetWindGust"]!=0]
subset_met_df

In [ ]:
array = subset_met_df.values
X = array[:, 1:8]
y = array[:, 0]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = StandardScaler()
rescaled_X_train = (scaler.fit_transform(X_train[:, 3:8])) # doesn't include lon, lat
rescaled_X_test = (scaler.fit_transform(X_test[:, 3:8])) # doesn't include lon, lat

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001,
                   power_t=0.5, max_iter=1500, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp = mlp.fit(rescaled_X_train, y_train)

y_predict_mlp = mlp.predict(rescaled_X_test)

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(X_test[:, 0], X_test[:, 1], latlon=True, c=y_predict_mlp, cmap='Purples',
          vmin=0, vmax=10, s=5)

plt.colorbar(label='predicted gusts (meter/sec)', extend='max')

plt.show()

In [ ]:
hist_nn = plt.hist(y_predict_mlp, bins=30, color='orange', range=[0,30])
plt.title("Neural Net Predictions")
plt.ylim(0,200)

In [ ]:
plt.scatter(X_test[:, 0], y_test,  color='navy', s=5, alpha=1)
plt.scatter(X_test[:, 0], y_predict_mlp, color='orange', s=5, alpha=0.3)
plt.title("Scatter Comparison")
plt.xlabel("Longitude", size = 10)
plt.ylabel("Wind Gust (meter/sec)", size = 10)

# Format HRRR data

In [ ]:
hrrr_lon = Hrrr.variables['gridlon_0']
hrrr_lat = Hrrr.variables['gridlat_0']
hrrr_gust = Hrrr.variables['GUST_P0_L1_GLC0']

hrrr_lon = np.reshape(hrrr_lon, 1905141)
hrrr_lat = np.reshape(hrrr_lat, 1905141)
hrrr_gust = np.reshape(hrrr_gust, 1905141)

hrrr_lon = hrrr_lon[0:1903585:1009]
hrrr_lat = hrrr_lat[0:1903585:1009]
hrrr_gust = hrrr_gust[0:1903585:1009]

### HRRR Predictions

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='c',
            width=8E6, height=8E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
m.drawparallels(parallels, labels=[False, True, True, False])
meridians = np.arange(10., 351., 20.)
m.drawmeridians(meridians, labels=[True, False, False, True])

m.scatter(hrrr_lon, hrrr_lat, latlon=True, c=hrrr_gust,
          cmap='coolwarm', vmin=5, vmax=10, s=9)
plt.show()